In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230311']#, '20230305']
SESSION = ['SE_Lek1']#, 'SE_Lek1']
DRONE = ['P1D1']#, 'P2D4', 'P3D5', 'P3D6']

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [4]:
for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            
            internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
            files = glob.glob(internal_path + '/' + date + '_' + session + '_' + drone + '*_tracked.csv')
            
            for file in sorted(files):
                print(date, session, drone, os.path.basename(file))
                
                df = pd.read_csv(file)
                df['x'] = df['bb_left'] + df['bb_width']/2
                df['y'] = df['bb_top'] + df['bb_height']/2
                df['idx'] = df['track_id']

                df = df.loc[:,['frame', 'x', 'y', 'idx']]
                
                # Step 1: Identify anchor frames
                anchor_frames = hf.identify_anchor_frames(df, thresholds)

                # Step 2: Find the best anchor for each frame
                df = hf.find_best_anchor_for_each_frame_parallel(df, anchor_frames)

                # Step 3: Apply homography using the best anchor for each frame
                df, homography_matrices, rms_errors = hf.apply_homography_to_best_anchor(df)
                
                anchor_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_Anchored.csv')
                pickle_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_homographies.pkl')
                df.to_csv(os.path.join(os.path.dirname(file), anchor_filename), index=False, mode='w')
                with open(os.path.join(os.path.dirname(file), pickle_filename), 'wb') as f:
                    pickle.dump(homography_matrices, f)

20230311 SE_Lek1 P1D1 20230311_SE_Lek1_P1D1_DJI_0229_YOLO_tracked.csv
Initial identification of anchors takes  23.844830989837646
Finding the best anchors took 325.3493890762329
Applying homography to the best anchors takes  182.97860097885132
20230311 SE_Lek1 P1D1 20230311_SE_Lek1_P1D1_DJI_0230_YOLO_tracked.csv
Initial identification of anchors takes  25.628989219665527
Finding the best anchors took 330.18655705451965
Applying homography to the best anchors takes  221.73514819145203
20230311 SE_Lek1 P1D1 20230311_SE_Lek1_P1D1_DJI_0231_YOLO_tracked.csv
Initial identification of anchors takes  32.021151065826416
Finding the best anchors took 512.8644139766693
Applying homography to the best anchors takes  250.29593300819397
20230311 SE_Lek1 P1D1 20230311_SE_Lek1_P1D1_DJI_0232_YOLO_tracked.csv
Initial identification of anchors takes  28.440222024917603
Finding the best anchors took 299.23716592788696
Applying homography to the best anchors takes  219.2483389377594
20230311 SE_Lek1 P1D1 2